# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset 

import joblib


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
label = 'vfc_level2_3_name'

In [5]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required


subscription_id = '6971f5ac-8af1-446e-8034-05acea24681f'
resource_group = 'aml-quickstarts-202466'
workspace_name = 'quick-starts-ws-202466'

workspace = Workspace(subscription_id, resource_group, workspace_name)



In [6]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required.

validation_data = Dataset.get_by_name(workspace, name='test_project')
validataion_data = validation_data.to_pandas_dataframe()
train_data = Dataset.get_by_name(workspace, name='train_project')
train_data = train_data.to_pandas_dataframe()


In [22]:
import pandas as pd


validation_data = validataion_data.set_index('Column1')
featurecols = validation_data.columns.tolist()
featurecols.remove(label)
X_val = validation_data[featurecols]
y_val = validation_data[label]

train_data = train_data.set_index('Column1')

X_train = train_data[featurecols]
y_train = train_data[label]

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [23]:
automl_config = AutoMLConfig(
    experiment_timeout_hours=7.0,
    task= 'classification',
    primary_metric= 'average_precision_score_weighted',
    enable_early_stopping= True,
    featurization='off',
    X =X_train,
    y = y_train, 
    X_valid = X_val, 
    y_valid = y_val,
    enable_voting_ensemble=False,
    enable_stack_ensemble=False,
    allowed_models=['BernoulliNaiveBayes', 'KNN', 'RandomForest'],
    y_min= 100,
    y_max= 1200,)

In [24]:
# Choose a name for your CPU cluster
cluster_name = "theresa"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(workspace, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)



Found existing cluster, use it.

Running


In [25]:
# TODO: Submit your experiment
# choose a name for experiment
experiment_name = 'capstone_automl'

experiment=Experiment(workspace, experiment_name)

run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.



********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|100                           |All-round view : View/restric...|10092                                 |
+------------------------------+--------------------------------+-----

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [30]:
best_run, best_model = run.get_output()

In [34]:
from azureml.core.model import Model


model = best_run.register_model(model_name='automl_knn', 
                           model_path='outputs/model.pkl',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.19.1')
model.download(target_dir='.', exist_ok=True, exists_ok=None)

'model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
